<img src="http://oproject.org/tiki-download_file.php?fileId=8&display&x=450&y=128">
<img src="http://gfif.udea.edu.co/root/tmva/img/tmva_logo.gif">
## TMVA Deep Neural Network Example

In [1]:
TMVA::Tools::Instance();

### Open input and output files

In [2]:
TString infilename = "higgs.root";
TFile *input       = TFile::Open(infilename);
TMVA::DataLoader *loader = new TMVA::DataLoader("higgs");

TString outfilename = "TMVA_DNN.root";
TFile *output       = TFile::Open(outfilename, "RECREATE");

TMVA::Factory *factory   = new TMVA::Factory("TMVAClassification",
                                             output,
                                             "AnalysisType=Classification");

--- Factory                  : You are running ROOT Version: 6.07/07, Apr 1, 2016
--- Factory                  : 
--- Factory                  : _/_/_/_/_/ _|      _|  _|      _|    _|_|   
--- Factory                  :    _/      _|_|  _|_|  _|      _|  _|    _| 
--- Factory                  :   _/       _|  _|  _|  _|      _|  _|_|_|_| 
--- Factory                  :  _/        _|      _|    _|  _|    _|    _| 
--- Factory                  : _/         _|      _|      _|      _|    _| 
--- Factory                  : 
--- Factory                  : ___________TMVA Version 4.2.1, Feb 5, 2015
--- Factory                  : 


### Prepare training data

In [3]:
TTree *tsignal     = (TTree *) input->Get("TreeS");
TTree *tbackground = (TTree *) input->Get("TreeB");
Double_t signalWeight = 1.0;
Double_t backgroundWeight = 1.0;
loader->AddSignalTree    (tsignal,     signalWeight);
loader->AddBackgroundTree(tbackground, backgroundWeight);

--- DataSetInfo              : Dataset[higgs] : Added class "Signal"	 with internal class number 0
--- higgs                    : Add Tree TreeS of type Signal with 5829123 events
--- DataSetInfo              : Dataset[higgs] : Added class "Background"	 with internal class number 1
--- higgs                    : Add Tree TreeB of type Background with 5170877 events


### Add variables

In [4]:
loader->AddVariable("lepton_pT",'F');
loader->AddVariable("lepton_eta",'F');
loader->AddVariable("lepton_phi",'F');
loader->AddVariable("missing_energy_magnitude",'F');
loader->AddVariable("missing_energy_phi",'F');
loader->AddVariable("jet_1_pt",'F');
loader->AddVariable("jet_1_eta",'F');
loader->AddVariable("jet_1_phi",'F');
loader->AddVariable("jet_1_b_tag",'F');
loader->AddVariable("jet_2_pt",'F');
loader->AddVariable("jet_2_eta",'F');
loader->AddVariable("jet_2_phi",'F');
loader->AddVariable("jet_2_b_tag",'F');
loader->AddVariable("jet_3_pt",'F');
loader->AddVariable("jet_3_eta",'F');
loader->AddVariable("jet_3_phi",'F');
loader->AddVariable("jet_3_b_tag",'F');
loader->AddVariable("jet_4_pt",'F');
loader->AddVariable("jet_4_eta",'F');
loader->AddVariable("jet_4_phi",'F');
loader->AddVariable("jet_4_b_tag",'F');
loader->AddVariable("m_jj",'F');
loader->AddVariable("m_jjj",'F');
loader->AddVariable("m_lv",'F');
loader->AddVariable("m_jlv",'F');
loader->AddVariable("m_bb",'F');
loader->AddVariable("m_wbb",'F');
loader->AddVariable("m_wwbb",'F');

TString dataString = "nTrain_Signal=1000:"
                     "nTrain_Background=1000:"
                     "nTest_Signal=100:"
                     "nTest_Background=100:"
                     "SplitMode=Random:"
                     "NormMode=NumEvents:"
                     "!V";
loader->PrepareTrainingAndTestTree("", "", dataString);


--- higgs                    : Preparing trees for training and testing...


### Define network structure and training scheme

In [5]:
TString configString = "!H:V";
configString += ":VarTransform=G";
configString += ":ErrorStrategy=CROSSENTROPY";
configString += ":WeightInitialization=XAVIERUNIFORM";
TString layoutString = "Layout=RELU|100,RELU|50,RELU|10,LINEAR";

TString trainingString = "TrainingStrategy="
                         "LearningRate=0.1,"
                         "Momentum=0.5,"
                         "Repetitions=1,"
                         "ConvergenceSteps=300,"
                         "BatchSize=20,"
                         "Regularization=None,"
                         "TestRepetitions=15,"
                         "Multithreading=True";

configString += ":" + layoutString + ":" + trainingString;
TMVA::Types::EMVA method = TMVA::Types::kDNN;
factory->BookMethod(loader, method, "DNN", configString);

--- Factory                  : Booking method: DNN DataSet Name: higgs
--- <VERBOSE> Configurable             : Parsing option string: 
--- <VERBOSE> Configurable             : ... "!H:V:VarTransform=G:ErrorStrategy=CROSSENTROPY:WeightInitialization=XAVIERUNIFORM:Layout=RELU|100,RELU|50,RELU|10,LINEAR:TrainingStrategy=LearningRate=0.1,Momentum=0.5,Repetitions=1,ConvergenceSteps=300,BatchSize=20,Regularization=None,TestRepetitions=15,Multithreading=True"
--- <VERBOSE> Configurable             : The following options are set:
--- <VERBOSE> Configurable             : - By User:
--- <VERBOSE> Configurable             :     <none>
--- <VERBOSE> Configurable             : - Default:
--- <VERBOSE> Configurable             :     Boost_num: "0" [Number of times the classifier will be boosted]
--- <VERBOSE> DNN                      : Parsing option string: 
--- <VERBOSE> DNN                      : ... "!H:V:VarTransform=G:ErrorStrategy=CROSSENTROPY:WeightInitialization=XAVIERUNIFORM:Layout=RELU|

### Train and evaluate method

In [6]:
factory->TrainAllMethods();
factory->TestAllMethods();
factory->EvaluateAllMethods();

--- Factory                  :  
--- Factory                  : Train all methods for Classification ...
--- DataSetFactory           : Dataset[higgs] : Splitmode is: "RANDOM" the mixmode is: "SAMEASSPLITMODE"
--- DataSetFactory           : Dataset[higgs] : Create training and testing trees -- looping over class "Signal" ...
--- DataSetFactory           : Dataset[higgs] : Weight expression for class 'Signal': ""
--- DataSetFactory           : Dataset[higgs] : Create training and testing trees -- looping over class "Background" ...
--- DataSetFactory           : Dataset[higgs] : Weight expression for class 'Background': ""
--- DataSetFactory           : Dataset[higgs] : Number of events in input trees (after possible flattening of arrays):
--- DataSetFactory           : Dataset[higgs] :     Signal          -- number of events       : 5829123  / sum of weights: 5.82912e+06
--- DataSetFactory           : Dataset[higgs] :     Background      -- number of events       : 5170877  / sum of we


learning rate reduced to 0.025


copying weights: -0.221571 | -0.221571
copying weights: -0.133096 | -0.133096
copying weights: -0.22788 | -0.22788
copying weights: -0.122102 | -0.122102
copying weights: 0.0814608 | 0.0814608
copying weights: 0.149409 | 0.149409
copying weights: 0.141894 | 0.141894
copying weights: 0.149271 | 0.149271
copying weights: 0.161608 | 0.161608
copying weights: 0.24589 | 0.24589
copying weights: 0.174261 | 0.174261
copying weights: -0.291056 | -0.291056
copying weights: 0.215818 | 0.215818
copying weights: 0.385786 | 0.385786
copying weights: 0.108685 | 0.108685
copying weights: 0.313237 | 0.313237
copying weights: -0.141279 | -0.141279
copying weights: 0.294856 | 0.294856
copying weights: -0.15564 | -0.15564
copying weights: 0.307646 | 0.307646
copying weights: 0.073652 | 0.073652
copying weights: 0.2523 | 0.2523
copying weights: 0.291408 | 0.291408
copying weights: 0.423833 | 0.423833
copying weights: -0.238475 | -0.238475
copying weights: -0.0451961 | -0.0451961
copying weights: -0.291829

--- DNN                      : Write monitoring histograms to file: TMVA_DNN.root:/higgs/Method_DNN/DNN
--- Factory                  : Training finished
--- Factory                  : 
--- Factory                  : Ranking input variables (method specific)...
--- DNN                      : Ranking result (top variable is best ranked)
--- DNN                      : -------------------------------------------------
--- DNN                      : Rank : Variable                 : Importance
--- DNN                      : -------------------------------------------------
--- DNN                      :    1 : lepton_pT                : 1.000e+00
--- DNN                      :    2 : lepton_eta               : 1.000e+00
--- DNN                      :    3 : lepton_phi               : 1.000e+00
--- DNN                      :    4 : missing_energy_magnitude : 1.000e+00
--- DNN                      :    5 : missing_energy_phi       : 1.000e+00
--- DNN                      :    6 : jet_1_pt    

--- <WARNING> RootFinder               : <Root> initial interval w/o root: (a=0.5, b=0.5), (Eff_a=0, Eff_b=0), (fa=-0.005, fb=-0.005), refValue = 0.005
--- <WARNING> RootFinder               : <Root> initial interval w/o root: (a=0.5, b=0.5), (Eff_a=0, Eff_b=0), (fa=-0.015, fb=-0.015), refValue = 0.015
--- <WARNING> RootFinder               : <Root> initial interval w/o root: (a=0.5, b=0.5), (Eff_a=0, Eff_b=0), (fa=-0.025, fb=-0.025), refValue = 0.025
--- <WARNING> RootFinder               : <Root> initial interval w/o root: (a=0.5, b=0.5), (Eff_a=0, Eff_b=0), (fa=-0.035, fb=-0.035), refValue = 0.035
--- <WARNING> RootFinder               : <Root> initial interval w/o root: (a=0.5, b=0.5), (Eff_a=0, Eff_b=0), (fa=-0.045, fb=-0.045), refValue = 0.045
--- <WARNING> RootFinder               : <Root> initial interval w/o root: (a=0.5, b=0.5), (Eff_a=0, Eff_b=0), (fa=-0.055, fb=-0.055), refValue = 0.055
--- <WARNING> RootFinder               : <Root> initial interval w/o root: (a=0.5, b=0.5

### Plot results

In [7]:
%jsroot
TCanvas *c=factory->GetROCCurve(loader);
c->Draw();